# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [3]:
!pip install gensim

     |████████████████████████████████| 23.9 MB 18.5 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 6.0 MB/s  eta 0:00:01


In [4]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [6]:
df = pd.read_json('News_Category_Dataset_v2.zip', lines=True)

In [7]:
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [12]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [17]:
pd.set_option('display.max_colwidth', 1000)

In [18]:
data[:5]

0                             [There, Were, 2, Mass, Shootings, In, Texas, Last, Week, ,, But, Only, 1, On, TV, She, left, her, husband, ., He, killed, their, children, ., Just, another, day, in, America, .]
1                                                                                 [Will, Smith, Joins, Diplo, And, Nicky, Jam, For, The, 2018, World, Cup, 's, Official, Song, Of, course, it, has, a, song, .]
2                                           [Hugh, Grant, Marries, For, The, First, Time, At, Age, 57, The, actor, and, his, longtime, girlfriend, Anna, Eberstein, tied, the, knot, in, a, civil, ceremony, .]
3                    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schiff, And, Democrats, In, New, Artwork, The, actor, gives, Dems, an, ass-kicking, for, not, fighting, hard, enough, against, Donald, Trump, .]
4    [Julianna, Margulies, Uses, Donald, Trump, Poop, Bags, To, Pick, Up, After, Her, Dog, The, ``, Dietland, '', actress, said, using, the, bags, is, a, ``, really, ca

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [20]:
model = Word2Vec(data, vector_size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [21]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55566338, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [22]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [23]:
wv.most_similar('Texas')

[('Ohio', 0.821323573589325),
 ('Louisiana', 0.8071303367614746),
 ('Pennsylvania', 0.8016950488090515),
 ('Massachusetts', 0.8013485670089722),
 ('Oregon', 0.8013160824775696),
 ('Utah', 0.7831077575683594),
 ('Georgia', 0.7778566479682922),
 ('Maryland', 0.7774062156677246),
 ('Connecticut', 0.7769040465354919),
 ('Florida', 0.7753750085830688)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [24]:
wv.most_similar('Texas', 'negative')

[('Burnings', 0.44955018162727356),
 ('Need-Based', 0.4416404962539673),
 ('rentier', 0.4336879849433899),
 ('Self-Interest', 0.432204008102417),
 ('Utermohlen', 0.42950692772865295),
 ('Hishammuddin', 0.4247688055038452),
 ('Dessy', 0.42238983511924744),
 ('finance', 0.4218912124633789),
 ('linkages', 0.4214991331100464),
 ('investment', 0.42112845182418823)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [25]:
wv['Texas']

array([-1.19571134e-01, -3.62673891e-03, -1.58694595e-01,  4.83537138e-01,
        1.34425247e+00, -9.24843669e-01, -7.76305616e-01,  1.68631685e+00,
       -9.69334602e-01,  1.31044388e+00, -2.63951159e+00, -8.79112184e-01,
        1.08851540e+00, -9.38403606e-01, -1.72762477e+00, -1.12213743e+00,
       -2.61844993e-01,  3.12854916e-01, -1.12262797e+00, -7.71365643e-01,
       -2.85581613e+00,  7.94874653e-02, -4.55963552e-01, -1.14857495e+00,
        5.64887583e-01,  1.48586917e+00, -1.36551678e+00, -2.01607203e+00,
       -8.45062077e-01,  4.77221966e-01,  1.27036348e-01, -2.38618135e-01,
       -1.35486114e+00, -1.97273767e+00, -2.13363208e-02, -4.33934003e-01,
       -1.36916482e+00, -1.02643955e+00, -1.85613298e+00, -2.51551008e+00,
       -1.35615778e+00,  6.52001679e-01, -1.15751278e+00,  1.33610725e-01,
        3.20452380e+00,  7.17229724e-01, -1.54897916e+00, -6.54149532e-01,
       -4.88828510e-01, -2.49642277e+00, -6.58194780e-01, -7.30763078e-01,
       -1.05940735e+00,  

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [26]:
wv.vectors

array([[ 3.1629808e-02,  1.6448522e+00,  7.6044077e-01, ...,
         3.9732188e-01, -7.4006158e-01, -1.5672743e-01],
       [ 6.4278007e-01,  1.4227149e+00,  6.7697421e-02, ...,
         1.4703852e+00,  8.5641921e-01, -1.1332139e-01],
       [ 1.9046861e+00,  3.7815645e-01,  1.6414464e+00, ...,
         2.0852416e+00,  4.9207735e-01, -9.5126903e-01],
       ...,
       [ 8.9077611e-04,  4.0586074e-03,  1.2541923e-01, ...,
         8.2866326e-02, -3.5003945e-02,  3.5081364e-02],
       [ 5.9503415e-03,  7.1250655e-02, -4.1008893e-02, ...,
         2.7638288e-02,  9.6572369e-02,  4.0632825e-02],
       [ 9.1096023e-03,  4.9868800e-02,  1.3501175e-02, ...,
        -1.2100897e-01, -2.3506436e-02, -6.6583599e-03]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [30]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('revival', 0.6895129084587097),
 ('princess', 0.6779900789260864),
 ('queen', 0.6398405432701111),
 ('villain', 0.6351662874221802),
 ('title', 0.6240376830101013),
 ('reminiscent', 0.6134358644485474),
 ('brunette', 0.6020019054412842),
 ('symbol', 0.5978144407272339),
 ('masterpiece', 0.5887969732284546),
 ('crown', 0.5869082808494568)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!